In [2]:
import pandas as pd
import numpy as np
from rsome import ro
from rsome import grb_solver as grb

## Data Ingestion

In [3]:
separation_time = pd.read_csv("data - Separation Time.csv",header=None).head(10).values
separation_time

array([[99999,     1,     3,     3,     3,     3,     3,     3,     3,
            3,     1,     1,     3,     3,     1],
       [    1, 99999,     3,     3,     3,     3,     3,     3,     3,
            3,     1,     1,     3,     3,     1],
       [    3,     3, 99999,     2,     2,     2,     2,     2,     2,
            2,     3,     3,     2,     2,     3],
       [    3,     3,     2, 99999,     2,     2,     2,     2,     2,
            2,     3,     3,     2,     2,     3],
       [    3,     3,     2,     2, 99999,     2,     2,     2,     2,
            2,     3,     3,     2,     2,     3],
       [    3,     3,     2,     2,     2, 99999,     2,     2,     2,
            2,     3,     3,     2,     2,     3],
       [    3,     3,     2,     2,     2,     2, 99999,     2,     2,
            2,     3,     3,     2,     2,     3],
       [    3,     3,     2,     2,     2,     2,     2, 99999,     2,
            2,     3,     3,     2,     2,     3],
       [    3,     3,   

In [4]:
landing_time_cost_df = pd.read_csv("data - Landing Time.csv").head(10)
landing_time_cost_df

,Aircraft ID,Earliest Landing Slot,Target Landing Slot,Latest Landing Slot,Penalty cost per time slot of landing before target,Penalty cost per time slot of landing after target,Size
0,0,3,4,12,10,10,3
1,1,4,6,15,10,10,3
2,2,2,2,11,30,30,2
3,3,2,2,11,30,30,2
4,4,3,3,11,30,30,2
5,5,3,3,12,30,30,2
6,6,3,3,12,30,30,1
7,7,3,3,11,30,30,1
8,8,3,3,12,30,30,1
9,9,3,4,13,30,30,1


In [5]:
earliest_landing = landing_time_cost_df["Earliest Landing Slot"].values
target_landing = landing_time_cost_df["Target Landing Slot"].values
latest_landing = landing_time_cost_df["Latest Landing Slot"].values

early_landing_penalty = landing_time_cost_df["Penalty cost per time slot of landing before target"].values
late_landing_penalty = landing_time_cost_df["Penalty cost per time slot of landing after target"].values
aircraft_size = landing_time_cost_df["Size"].values

In [6]:
N = len(earliest_landing)
N

10

In [7]:
runway_max_size = pd.read_csv("data - Runway.csv")['Max Size'].values
runway_max_size

array([3, 1, 2], dtype=int64)

In [8]:
num_runways = runway_max_size.shape[0]
num_runways

3

In [9]:
max_aircraft_runway_size_diff = max(aircraft_size.max() -  runway_max_size.min(),  runway_max_size.max() - aircraft_size.min())
max_aircraft_runway_size_diff

2

In [10]:
# good_weather_array = pd.read_csv("data - Weather.csv")['Thunderstorm'].values
thunderstorm_array = np.array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
thunderstorm_array
T = len(thunderstorm_array)

## Model

In [11]:
model = ro.Model("Aircaft Landing")

### Decision Variables

In [18]:
scheduled_landing = model.dvar(N, vtype="i")
landing_earliness = model.dvar(N)
landing_lateness = model.dvar(N)

#NxN Matrix to determine whether plane i lands before plane j, i != j
i_lands_before_j = model.dvar((N,N), "B")

# NxN Matrix to determine whether plane i lands before plane j, i != j
lands_on_same_runway = model.dvar((N,N), "B")

# Nxnum_runways Matrix to determine whether plane i lands on runway r
runway_allocation = model.dvar((N,num_runways), "B")

# Nxnum_runways Matrix to determine whether plane i lands on runway r
timeslot_allocation = model.dvar((N,T), "B")

### Objective Function

In [13]:
model.min(sum(early_landing_penalty*landing_earliness) + sum(late_landing_penalty*landing_lateness))

### Constraints

In [25]:
# Aircraft has to land between the latest and earliest possible time
for i in range(len(latest_landing)):
    # model.st(earliest_landing[i] <= scheduled_landing[i] <= latest_landing[i])
    model.st(earliest_landing[i] <= scheduled_landing[i])
    model.st(scheduled_landing[i] <= latest_landing[i])

# Calculating the earliness & lateness
# Since landing_earliness, landing_lateness > 0
# If scheduled_landing - target_landing >= 0, landing_earliness >=0 & landing_lateness ==0
# If scheduled_landing - target_landing <= 0, landing_earliness ==0 & landing_lateness >=0
model.st(target_landing - scheduled_landing  == landing_earliness - landing_lateness)

M = 99999
for j in range(N):
    for i in range(N):
        if i != j:
            # If 2 planes are scheduled for the same runway, then at least S amt of time should have elapsed between plane i and plane j
            model.st(scheduled_landing[j] - scheduled_landing[i] >= 
                     separation_time[i,j]*lands_on_same_runway[i,j] - M*i_lands_before_j[j,i])
            
            # Every 2 plane must land either before or after and not the same time
            model.st(i_lands_before_j[i,j] + i_lands_before_j[j,i] == 1)
            
            # Defining the link between lands_on_same_runway & runway_allocation
            for r in range(num_runways):
                model.st(lands_on_same_runway[i,j] >= runway_allocation[i,r] + runway_allocation[j,r] - 1)

# Aircraft size should not be larger than max size of runway
for aircraft_index in range(N):
    for runway_index in range(num_runways):
        model.st(aircraft_size[aircraft_index] - runway_max_size[runway_index] <= (1 - runway_allocation[aircraft_index,runway_index])*max_aircraft_runway_size_diff)


# Each aircraft can only land on 1 runway
for i in range(N):
    model.st(sum(runway_allocation[i,r] for r in range(num_runways)) == 1)

    model.st(sum(timeslot_allocation[i,t] for t in range(T)) == 1)

# Each timeslot will be unavailable during thunderstorms
for t, thunderstorm in enumerate(thunderstorm_array):
    if thunderstorm:
        model.st(sum(timeslot_allocation[i,t] for i in range(N)) == 1)

# Linking the scheduled_landing variable to the timeslot_allocation variable
for plane_index, landing in enumerate(scheduled_landing):
    model.st(timeslot_allocation[plane_index][landing-1]  == 1)


# Ensure that the model does not schedule any landing beyond the timing for which we have thunderstorm information
model.st(scheduled_landing <= thunderstorm_array.shape[0])

# Non negative Constraints
model.st(scheduled_landing >= 0,
        landing_earliness >= 0,
        landing_lateness >= 0
        )

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
model.do_math()

Conic program object:
Number of variables:           261
Continuous/binaries/integers:  31/230/0
---------------------------------------------
Number of linear constraints:  501
Inequalities/equalities:       391/110
Number of coefficients:        1461
---------------------------------------------
Number of SOC constraints:     0
---------------------------------------------
Number of ExpCone constraints: 0

### Solving

In [ ]:
model.solve(grb)

Being solved by Gurobi...
Solution status: 2
Running time: 11.8920s


In [ ]:
print("Runway allocation:", runway_allocation.get())

Runway allocation: [[1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [ ]:
runway_allocation.get()

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [ ]:
np.argmax(runway_allocation.get(), axis=1) + 1

array([1, 1, 3, 1, 3, 1, 2, 3, 2, 2], dtype=int64)

In [ ]:
print("Scheduled landing times:", scheduled_landing.get())

Scheduled landing times: [8. 9. 5. 5. 7. 3. 3. 3. 5. 7.]


In [ ]:
print("Earliness:", landing_earliness.get())
print("Lateness:", landing_lateness.get())

Earliness: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Lateness: [4. 3. 3. 3. 4. 0. 0. 0. 2. 3.]


In [ ]:
print("Total cost:", model.get())

Total cost: 520.0


In [ ]:
model_output = pd.DataFrame(
    {
    'Aircraft ID' : landing_time_cost_df['Aircraft ID'].values,
    'Runway' :  np.argmax(runway_allocation.get(), axis=1) + 1,
    'Earliest Landing' : landing_time_cost_df['Earliest Landing Slot'].values,
    'scheduled_landing': scheduled_landing.get(),
    'Latest Landing' : landing_time_cost_df['Latest Landing Slot'].values,
    'landing_earliness': landing_earliness.get(),
    'landing_lateness' : landing_lateness.get()
    }).sort_values(['Runway','scheduled_landing']).reset_index(drop=True)
model_output

,Aircraft ID,Runway,Earliest Landing,scheduled_landing,Latest Landing,landing_earliness,landing_lateness
0,5,1,3,3.0,12,0.0,0.0
1,3,1,2,5.0,11,0.0,3.0
2,0,1,3,8.0,12,0.0,4.0
3,1,1,4,9.0,15,0.0,3.0
4,6,2,3,3.0,12,0.0,0.0
5,8,2,3,5.0,12,0.0,2.0
6,9,2,3,7.0,13,0.0,3.0
7,7,3,3,3.0,11,0.0,0.0
8,2,3,2,5.0,11,0.0,3.0
9,4,3,3,7.0,11,0.0,4.0


# Wrapping the Model as a Function

In [ ]:
def get_optimal_landing(no_of_runways, separation_time, earliest_landing, target_landing, latest_landing, 
                        early_penalty, late_penalty):
    '''
    Prints the optimal landing time and runway allocation for each plane, among other variables.
    
    Parameters:
        no_of_runways (int): the number of available runways
        separation_time (numpy.ndarray): a matrix of separation times when planes land on the same runway
        earliest_landing (numpy.ndarray): an array of the earliest time each plane can land
        target_landing (numpy.ndarray): an array of the target landing time for each plane
        latest_landing (numpy.ndarray): an array of the latest time each plane can land
        early_penalty (numpy.ndarray): an array containing the penalty per unit time of landing each plane early
        late_penalty (numpy.ndarray): an array containing the penalty per unit time of landing each plane late
    '''
    
    check_input_shapes(separation_time, len(earliest_landing), len(target_landing), len(latest_landing), 
                       len(early_penalty), len(late_penalty))
    
    no_of_planes = len(earliest_landing)
    
    model = ro.Model('Airplane Landing')
    
    # Decision variables
    scheduled_landing_time = model.dvar(no_of_planes)
    earliness = model.dvar(no_of_planes)
    lateness = model.dvar(no_of_planes)
    lands_before = model.dvar((no_of_planes, no_of_planes), "B")
    lands_on_same_runway = model.dvar((no_of_planes, no_of_planes), "B")
    runway_allocation = model.dvar((no_of_planes, no_of_runways), "B")
    
    # Objective
    model.min(sum(early_penalty*earliness) + sum(late_penalty*lateness))
    
    # Constraints
    model.st(earliest_landing <= scheduled_landing_time <= latest_landing)
    model.st(scheduled_landing_time - target_landing == earliness - lateness)
    M = 99999
    for j in range(no_of_planes):
        for i in range(no_of_planes):
            if i != j:
                model.st(scheduled_landing_time[j] - scheduled_landing_time[i] >= 
                         separation_time[i,j]*lands_on_same_runway[i,j] - M*lands_before[j,i])
    for j in range(no_of_planes):
        for i in range(no_of_planes):
            if i != j:
                model.st(lands_before[i,j] + lands_before[j,i] == 1)
    for j in range(no_of_planes):
        for i in range(no_of_planes):
            if i != j:
                for r in range(no_of_runways):
                    model.st(lands_on_same_runway[i,j] >= runway_allocation[i,r] + runway_allocation[j,r] - 1)
    for i in range(no_of_planes):
        model.st(sum(runway_allocation[i,r] for r in range(no_of_runways)) == 1)
    
    model.st(scheduled_landing_time >= 0,
             earliness >= 0,
             lateness >= 0)
    
    # Get solution
    model.solve(grb)
    
    # Print results
    print("-------------------------")
    print("Scheduled landing times:", scheduled_landing_time.get())
    print("Earliness:", earliness.get())
    print("Lateness:", lateness.get())
    print("Whether i lands before j:\n", lands_before.get())
    print("Whether i lands on the same runway as j:\n", lands_on_same_runway.get())
    print("Runway allocation:\n", runway_allocation.get())
    
    landing_time_to_plane = {v:k for k, v in enumerate(scheduled_landing_time.get())}
    print("Sequence of landing:", [landing_time_to_plane[time] for time in sorted(scheduled_landing_time.get())])
    print("Sequence of landing times:", sorted(scheduled_landing_time.get()))
    print("Total cost:", model.get())
    
    
def check_input_shapes(separation, *args):
    assert len(set(args)) == 1, "Length of inputs must be equal"
    assert separation.shape == (args[0], args[0]), "Shape of separation must be NxN, where N is the length of each of the other inputs"

In [ ]:
get_optimal_landing(num_runways, separation_time, earliest_landing, target_landing, latest_landing, early_landing_penalty, late_landing_penalty)

Being solved by Gurobi...
Solution status: 2
Running time: 0.0187s
-------------------------
Scheduled landing times: [155. 258.  98. 106. 123. 132. 138. 140. 150. 180.]
Earliness: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Lateness: [0. 0. 0. 0. 0. 3. 0. 0. 0. 0.]
Whether i lands before j:
 [[0. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 0. 0. 1. 1. 1. 1. 1. 1.]
 [1. 1. 0. 0. 0. 1. 1. 1. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1. 1. 1.]
 [1. 1. 0. 0. 0. 0. 0. 1. 1. 1.]
 [1. 1. 0. 0. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Whether i lands on the same runway as j:
 [[0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 0. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 0. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 0. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 0. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 0. 0. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 0. 0. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 0. 1. 1.]
 [0. 1. 1. 1. 1. 1. 1. 1. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]]
Runway allocat

# Appendix